In [2]:
import psycopg2
from psycopg2 import sql, OperationalError, Error
from typing import Optional, Any, List, Dict, Union

class PostgresConnection:
    def __init__(self, host: str, user: str, password: str, database: str = None, port: int = 5432):
        self.__host = host
        self.__user = user
        self.__password = password
        self.__database = database
        self.__port = port
        self.__connection: Optional[Any] = None
        self.__cursor: Optional[Any] = None

    def connect(self) -> None:
        try:
            self.__connection = psycopg2.connect(
                host=self.__host,
                user=self.__user,
                password=self.__password,
                database=self.__database if self.__database else None,
                port=self.__port
            )
            self.__cursor = self.__connection.cursor()
        except OperationalError as err:
            if self.__database and 'database "{}" does not exist'.format(self.__database) in str(err):
                self.create_database()
                self.__connection = psycopg2.connect(
                    host=self.__host,
                    user=self.__user,
                    password=self.__password,
                    database=self.__database,
                    port=self.__port
                )
                self.__cursor = self.__connection.cursor()
            else:
                raise Exception(f"Error connecting to the database: {err}")

    def disconnect(self) -> None:
        try:
            if self.__cursor:
                self.__cursor.close()
            if self.__connection:
                self.__connection.close()
        except OperationalError as err:
            raise Exception(f"Error disconnecting from the database: {err}")

    def create_database(self) -> None:
        try:
            temp_connection = psycopg2.connect(
                host=self.__host,
                user=self.__user,
                password=self.__password,
                port=self.__port
            )
            temp_connection.autocommit = True
            temp_cursor = temp_connection.cursor()
            temp_cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(self.__database)))
            temp_cursor.close()
            temp_connection.close()
        except OperationalError as err:
            raise Exception(f"Failed to create database: {err}")

    def create_table(self, table_name: str, columns: Dict[str, str]) -> None:
        try:
            self.connect()
            columns_str = ', '.join([f'{col} {data_type}' for col, data_type in columns.items()])
            create_table_query = f'CREATE TABLE {table_name} ({columns_str})'
            self.__cursor.execute(create_table_query)
            self.__connection.commit()
        except OperationalError as err:
            raise Exception(f"Failed to create table: {err}")
        finally:
            self.disconnect()

    def insert_record(self, table_name: str, record: Dict[str, Any]) -> None:
        try:
            self.connect()
            columns = ', '.join(record.keys())
            values = tuple(record.values())
            placeholders = ', '.join(['%s'] * len(record))
            insert_query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
            self.__cursor.execute(insert_query, values)
            self.__connection.commit()
        except OperationalError as err:
            raise Exception(f"Failed to insert record: {err}")
        finally:
            self.disconnect()

    def select_record(self, table_name: str, conditions: str = None) -> List[List[Any]]:
        try:
            self.connect()
            select_query = f'SELECT * FROM {table_name}'
            if conditions:
                select_query += f' WHERE {conditions}'
            self.__cursor.execute(select_query)
            records = self.__cursor.fetchall()
            return [list(record) for record in records]  # Convert tuple to list
        except OperationalError as err:
            raise Exception(f"Failed to select record: {err}")
        finally:
            self.disconnect()

    def update_record(self, table_name: str, record: Dict[str, Any], conditions: str) -> None:
        try:
            self.connect()
            set_clause = ', '.join([f'{key}=%s' for key in record.keys()])
            values = tuple(record.values())
            update_query = f'UPDATE {table_name} SET {set_clause} WHERE {conditions}'
            self.__cursor.execute(update_query, values)
            self.__connection.commit()
        except OperationalError as err:
            raise Exception(f"Failed to update record: {err}")
        finally:
            self.disconnect()

    def delete_record(self, table_name: str, conditions: str) -> None:
        try:
            self.connect()
            delete_query = f'DELETE FROM {table_name} WHERE {conditions}'
            self.__cursor.execute(delete_query)
            self.__connection.commit()
        except OperationalError as err:
            raise Exception(f"Failed to delete record: {err}")
        finally:
            self.disconnect()

ModuleNotFoundError: No module named 'psycopg2'